# Separating Administrative and Business Logic in Python


---

### Evgeny Demchenko

* Twitter: [@littlepea](https://twitter.com/littlepea12)
* GitHub: [littlepea](https://github.com/littlepea/)

# Outline

* Types of software logic/functionality?
* Why separate business logic from the rest?
* Practical examples

# Types of logic

## Business/domain logic

* Core functionality of your application
* Domain-specific rules and entities

## Administrative logic

* I/O
* Caching
* Making network requests
* etc...

## Interface logic

* Handling user input
* Displaying/rendering output

## Application logic

* Use cases
* Workflows
* Glue code

# Why separate business logic from the rest?

* The core business logic should not depend on the framework or delivery mechanisms (Web, CLI, etc...)
* Business logic should be easy to unit test without I/O or mocking
* Business logic changes at different times and for different reasons than the rest

# Example

## Air Quality checking script

Let's say we want to implement a simple air quality checking script that for a specific city will tell us if the air is "Good", Unhealthy", "Hazardous", etc:

```commandline
$ python air_quality.py Beijing

Checking air quality for Beijing...

The air in Beijing is Moderate (98.5)
Feel free to go out!
```

# Requirements:

* Based on [Open AQ Platform API](https://docs.openaq.org/)
* City is required
* AQI for a city should be an average of PM2.5 for all stations
* Levels are based on [World Air Quality Index project's scale](http://aqicn.org/scale/)
* For Hazardous pollution, need to stay indoors
* For Unhealthy levels, should wear a mask
* Otherwise, can go outside

## Air Quality Index Scale

![](https://www.evernote.com/l/AHRRR4lTCIRE2oL_Tm4hopK8beFBpN-wgQcB/image.png)

# Initial implementation

[air_quality.py](https://github.com/littlepea/python-admin-business-logic-talk/blob/master/before/air_quality.py)

In [1]:
# Let's create some useful constants

from collections import OrderedDict


API_BASE = 'https://api.openaq.org/v1/latest'

AQI_LEVELS = OrderedDict()
AQI_LEVELS[51] = {
    'range': '0-50',
    'level': 'Good'
}
AQI_LEVELS[101] = {
    'range': '51-100',
    'level': 'Moderate'
}
AQI_LEVELS[151] = {
    'range': '101-150',
    'level': 'Unhealthy for Sensitive Groups'
}
AQI_LEVELS[201] = {
    'range': '151-200',
    'level': 'Unhealthy',
    'mask': True
}
AQI_LEVELS[301] = {
    'range': '201-300',
    'level': 'Very Unhealthy',
    'maks': True
}
AQI_LEVELS[10000] = {
    'range': '300+',
    'level': 'Hazardous',
    'indoors': True
}

CACHE = {}

In [2]:
# Main code of the application

import urllib2
import json

def main(city):
    print('\nChecking air quality for {}...\n'.format(city))

    # Check cache
    stations = CACHE.get(city)

    if not stations:
        try:
            # Call the API
            url = '{}?city={}'.format(API_BASE, city)
            response = urllib2.urlopen(url)
            results = json.load(response)
            stations = results['results']

            # Write to cache
            CACHE[city] = stations
        except urllib2.HTTPError, e:
            print('Could not retrieve results from the server: {}'.format(e))
            return

    if not stations:
        print('No stations found in {}'.format(city))
        return

    # Get PM2.5 values from all stations
    pm25 = [
        measurement['value']
        for station in stations
        for measurement in station['measurements']
        if measurement['parameter'] == 'pm25'
    ]

    # Calculate average PM2.5
    average_pm25 = sum(pm25, 0.0) / len(pm25)

    # Find the right AQI level
    aqi = None
    for ceiling, level in AQI_LEVELS.items():
        if average_pm25 < ceiling:
            aqi = level
            break

    # Display the AQI level to the user
    print('The air in {} is {} ({})'.format(
        city,
        aqi['level'],
        average_pm25
    ))

    if aqi.get('indoors'):
        print('Please, stay indoors with purified air.')
    elif aqi.get('mask'):
        print('Please, wear a mask if going out.')
    else:
        print('Feel free to go out!')

In [3]:
# Run it!

main('Beijing')


Checking air quality for Beijing...

The air in Beijing is Unhealthy for Sensitive Groups (108.0)
Feel free to go out!


# Separating the Business Logic

In [4]:
from collections import OrderedDict, namedtuple


Station = namedtuple('Station', ['name', 'pm25'])
Level = namedtuple('Level', ['name', 'aqi', 'mask', 'indoors'])


AQI_LEVELS = OrderedDict()
AQI_LEVELS[51] = {
    'range': '0-50',
    'level': 'Good'
}
AQI_LEVELS[101] = {
    'range': '51-100',
    'level': 'Moderate'
}
AQI_LEVELS[151] = {
    'range': '101-150',
    'level': 'Unhealthy for Sensitive Groups'
}
AQI_LEVELS[201] = {
    'range': '151-200',
    'level': 'Unhealthy',
    'mask': True
}
AQI_LEVELS[301] = {
    'range': '201-300',
    'level': 'Very Unhealthy',
    'maks': True
}
AQI_LEVELS[10000] = {
    'range': '300+',
    'level': 'Hazardous',
    'indoors': True
}


def city_aqi(stations):
    pm25 = [station.pm25 for station in stations]
    return sum(pm25, 0.0) / len(pm25)


def aqi_level(aqi):
    for ceiling, level in AQI_LEVELS.items():
        if aqi < ceiling:
            return Level(
                name=level['level'],
                aqi=aqi,
                mask=level.get('mask', False),
                indoors=level.get('indoors', False))


## Now our main function becomes:

In [5]:
def main(city):
    print('\nChecking air quality for {}...\n'.format(city))

    # Check cache
    stations = CACHE.get(city, [])

    if not stations:
        try:
            # Call the API
            url = '{}?city={}'.format(API_BASE, city)
            response = urllib2.urlopen(url)
            results = json.load(response)
            for result in results['results']:
                pm25 = None
                for measurement in result['measurements']:
                    if measurement['parameter'] == 'pm25':
                        pm25 = measurement['value']

                stations.append(Station(
                    name=result['location'],
                    pm25=pm25))

            # Write to cache
            CACHE[city] = stations
        except urllib2.HTTPError, e:
            print('Could not retrieve results from the server: {}'.format(e))
            return

    if not stations:
        print('No stations found in {}'.format(city))
        return

    level = aqi_level(city_aqi(stations))  # Business logic !

    # Display the AQI level to the user
    print('The air in {} is {} ({})'.format(
        city,
        level.name,
        level.aqi
    ))

    if level.indoors:
        print('Please, stay indoors with purified air.')
    elif level.mask:
        print('Please, wear a mask if going out.')
    else:
        print('Feel free to go out!')

# Separating the Application Logic

In [6]:
API_BASE = 'https://api.openaq.org/v1/latest'
CACHE = {}


def _get_city_url(city):
    return '{}?city={}'.format(API_BASE, city)


def _load_results(url):
    try:
        response = urllib2.urlopen(url)
        results = json.load(response)
        return results['results']
    except urllib2.HTTPError, e:
        return []


def _get_station_pm25(station):
    for measurement in station['measurements']:
        if measurement['parameter'] == 'pm25':
            return measurement['value']


def get_stations(city):
    # Check cache
    stations = CACHE.get(city, [])

    if not stations:
        for result in _load_results(_get_city_url(city)):
            stations.append(Station(
                name=result['location'],
                pm25=_get_station_pm25(result)))

        # Write to cache
        CACHE[city] = stations

    return stations


def get_recommendation(level):
    if level.indoors:
        return 'Please, stay indoors with purified air.'

    if level.mask:
        return 'Please, wear a mask if going out.'

    return 'Feel free to go out!'

## Now our main function becomes:

In [7]:
def main(city):
    print('\nChecking air quality for {}...\n'.format(city))

    stations = get_stations(city)

    if not stations:
        print('No stations found in {}'.format(city))
        return

    level = aqi_level(city_aqi(stations))
    recommendation = get_recommendation(level)
    
    print('The air in {} is {} ({})'.format(city, level.name, level.aqi))
    print(recommendation)

# Separating the Administration Logic

In [8]:
CACHE = {}


def cache(func):
    def wrapper(*args):
        key = '{}_{}'.format(
            func.__name__,
            '-'.join(args)
        )
        value = CACHE.get(key) or func(*args)
        CACHE[key] = value
        return value

    return wrapper

## Now application logic is simpler:

In [9]:
@cache
def get_stations(city):
    return [
        Station(
            name=result['location'],
            pm25=_get_station_pm25(result))
        for result in _load_results(_get_city_url(city))
    ]

### Instead of:

In [10]:
def get_stations(city):
    # Check cache
    stations = CACHE.get(city, [])

    if not stations:
        for result in _load_results(_get_city_url(city)):
            stations.append(Station(
                name=result['location'],
                pm25=_get_station_pm25(result)))

        # Write to cache
        CACHE[city] = stations

    return stations

## Final modules structure:

* [app.py](https://github.com/littlepea/python-admin-business-logic-talk/blob/master/after/app.py) (Application Logic)
* [aqi.py](https://github.com/littlepea/python-admin-business-logic-talk/blob/master/after/aqi.py) (Business Logic)
* [cache.py](https://github.com/littlepea/python-admin-business-logic-talk/blob/master/after/cache.py) (Administration Logic)
* [cli.py](https://github.com/littlepea/python-admin-business-logic-talk/blob/master/after/cli.py) (Interface Logic)

Let's run it!

In [11]:
main('Beijing')


Checking air quality for Beijing...

The air in Beijing is Unhealthy for Sensitive Groups (108.0)
Feel free to go out!


# Architecture

This can be considered to be an extremely simplified example of the [Onion Architecture](https://dzone.com/articles/onion-architecture-is-interesting).

![](http://tidyjava.com/wp-content/uploads/2017/02/obrazek_2.png)

# Benefits

### Better Domain Modelling

The whole application is built on top of well-defined domain logic.
  
### Directed coupling

The most important code depends on nothing, everything depends on it.
  
### Flexibility

From the inner layer perspective you can swap anything in the outer layers and everything will still work fine.
  
### Testeability

The application core has no dependencies and can easily be tested in isolation.

# Testeability

Now we can test the Business Logic in isonation without mocking anything:

In [12]:
import unittest


class CityAqiTest(unittest.TestCase):
    def test_city_aqi(self):
        stations = [Station('s1', 100), Station('s2', 30), Station('s3', 20)]
        self.assertEqual(50, city_aqi(stations))


class AqiLevelTest(unittest.TestCase):
    def test_good_aqi_level(self):
        level = aqi_level(50)
        self.assertEqual('Good', level.name)
        self.assertFalse(level.mask)

    def test_unhealthy_aqi_level(self):
        level = aqi_level(180)
        self.assertEqual('Unhealthy', level.name)
        self.assertTrue(level.mask)

    def test_hazardous_aqi_level(self):
        level = aqi_level(800)
        self.assertEqual('Hazardous', level.name)
        self.assertTrue(level.indoors)
        

unittest.main(argv=['ignored', '-v'], exit=False)

test_good_aqi_level (__main__.AqiLevelTest) ... ok
test_hazardous_aqi_level (__main__.AqiLevelTest) ... ok
test_unhealthy_aqi_level (__main__.AqiLevelTest) ... ok
test_city_aqi (__main__.CityAqiTest) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.004s

OK


# Flexibility

Let's say we've decided to use median instead of average for city AQI:

In [13]:
def median(numbers):
    return (sorted(numbers)[int(round((len(numbers) - 1) / 2.0))] +
            sorted(numbers)[int(round((len(numbers) - 1) // 2.0))]) / 2.0


def city_aqi(stations):
    pm25 = [station.pm25 for station in stations]
    return median(pm25)

Then we update the tests and they still pass:

In [15]:
class CityAqiTest(unittest.TestCase):
    def test_city_aqi(self):
        stations = [Station('s1', 100), Station('s2', 30), Station('s3', 20)]
        self.assertEqual(30, city_aqi(stations))
        
unittest.main(argv=['ignored', '-v'], exit=False)

test_good_aqi_level (__main__.AqiLevelTest) ... ok
test_hazardous_aqi_level (__main__.AqiLevelTest) ... ok
test_unhealthy_aqi_level (__main__.AqiLevelTest) ... ok
test_city_aqi (__main__.CityAqiTest) ... ok

----------------------------------------------------------------------
Ran 4 tests in 0.004s

OK


And the main program still working:

In [17]:
main('Beijing')


Checking air quality for Beijing...

The air in Beijing is Unhealthy for Sensitive Groups (108.0)
Feel free to go out!


# Q & A

---

### Evgeny Demchenko

* Twitter: [@littlepea](https://twitter.com/littlepea12)
* GitHub: [littlepea](https://github.com/littlepea/)